In [69]:
import requests, pandas as pd
import re
from bs4 import BeautifulSoup

In [70]:
r= requests.get("https://en.wikipedia.org/wiki/List_of_air_operations_during_the_Battle_of_Europe")
c= r.content

soup= BeautifulSoup(c,"html.parser")

body= soup.find_all("p")
body= body[4:-1]  # This is where the meaningful data lies

In [71]:
# Make a function that converts the Date into numerical form
def DateConversion(date):
    # Split up the string into month and day
    try:
        month= date.split(" ")[0].capitalize()
        day= date.split(" ")[1]
    except:
        return
    # Then convert the month to an integer
    if month == "January": month= 1
    elif month == "February": month = 2
    elif month == "February": month = 2
    elif month == "March": month = 3
    elif month == "April": month = 4
    elif month == "May": month = 5
    elif month == "June": month = 6
    elif month == "July": month = 7
    elif month == "August": month = 8
    elif month == "September": month = 9
    elif month == "October": month = 10
    elif month == "November": month = 11
    elif month == "December": month = 12
    else:
        # This part will run if the day is listed first
        if day == "January": day= 1
        elif day == "February": day = 2
        elif day == "February": day = 2
        elif day == "March": day = 3
        elif day == "April": day = 4
        elif day == "May": day = 5
        elif day == "June": day = 6
        elif day == "July": day = 7
        elif day == "August": day = 8
        elif day == "September": day = 9
        elif day == "October": day = 10
        elif day == "November": day = 11
        elif day == "December": day = 12
        return [day, month]
    return [month, day]

In [72]:
# Make a function that reads which country is responsible for the attack based on the icon.
def CountryIcon(href):
    # Get the href of the image.
    if href == "/wiki/File:Regulation_WW_II_Underwing_Balkenkreuz.png": country= "Germany"
    elif href == "/wiki/File:Roundel_of_the_German_Air_Force_(with_Border).svg": country= "Germany"
    elif href == "/wiki/File:Balkenkreuz.svg": country= "Germany"
    elif href == "/wiki/File:Roundel_of_Poland_(1921-1993).svg": country= "Poland"
    elif href == "/wiki/File:RAF_roundel.svg": country= "United Kingdom"
    elif href == "/wiki/File:Red_star.svg": country= "Soviet Union"
    elif href == "/wiki/File:French-roundel.svg": country= "France"
    elif href == "/wiki/File:Italy-Royal-Airforce.svg": country= "Italy"
    elif href == "/wiki/File:US_Army_Air_Corps_Hap_Arnold_Wings.svg": country= "United States"
    elif href == "/wiki/File:82_ABD_SSI.svg": country= "United States"
    elif href == "/wiki/File:USA_-_Army_Field_Artillery_Insignia.png": country= "United States"
    else:
        country= "*********************************************"
    return country
                                    

In [73]:
# We also need a function that will remove the line breaks
def CleanText(text):
    # Split the string at the colon.  The part before the colon should be the date.  The second part is the text.
    # Split returns a list, so we join them back into one string.  Then, remove the first space and the \n ([1:-1]).
    string= " ".join(text.split(":", 1)[1:])[1:-1]
    string= re.sub("\[\d{1,2}\](:\d{1,2})?(-\d{1,3})?", "", string)
    return string

In [74]:
# The data we're looking for will contain the month, day, and year of the attack, the country initiating the attack,
# and whether or not the attack was successful.

data= [] # This will hold all the entries that need to go into the .csv

# There will be an initial entry added to data so that the first entry has something for comparison.
data.append({"Month":1, "Day":1, "Year":1939, "Country":"None", "Details":"Test"})

for entry in body:
    d= {} # For every entry, there will be a dictionary containing the relevant information
    
    # First, we extract the month, day, and year from an entry
    try:
        [month, day]= DateConversion(entry.b.string)
        d["Month"]= month
        d["Day"]= day
    except:
        # There is one entry that isn't bolded so doesn't get caught by the DateConversion.
        if data[-1]["Month"]== 6 and data[-1]["Day"]== "9" and data[-1]["Year"]== 1940:
                d["Month"]= 6
                d["Day"]= "11/12"
        # Some entries don't have a day, or last several months.  These enties must be addressed.
        if data[-1]["Month"]== 8 and data[-1]["Day"]== "24" and data[-1]["Year"]== 1942: continue
        if data[-1]["Month"]== 3 and data[-1]["Day"]== "6" and data[-1]["Year"]== 1944:
            d["Month"]= 3
            d["Day"]= "1-31"
        if data[-1]["Month"]== 3 and data[-1]["Day"]== "1-31" and data[-1]["Year"]== 1944: continue
        if data[-1]["Month"]== 3 and data[-1]["Day"]== "14" and data[-1]["Year"]== 1945:
            d["Month"]= 2
            d["Day"]= "1-28"
    # Now check if the month in question is lower than the previous entry.  If it is, that means it's the start of
    # a new year, and the year should be incremented.  If not, it retains the year of the previous entry.
    if d["Month"] < data[-1]["Month"]:
        d["Year"]= data[-1]["Year"] + 1
    else:
        d["Year"]= data[-1]["Year"]
    # Some entries are out of order and thus get the wrong Year.  This part corrects that problem for those entries.
    if d["Month"] == 9 and d["Day"] == "15" and d["Year"] == 1941: d["Year"] = 1940
    if d["Month"] == 2 and d["Day"] == "1-28" and d["Year"] == 1946: d["Year"] = 1945
        
    # Next, let's figure out which country initiated the attack.
    try:
        country= CountryIcon(entry.a["href"])
        d["Country"]= country
    except:
        # Some entries don't have an icon, but there was still an attack.  These entries require manual inputs.
        if d["Month"] == 5 and d["Day"] == "10" and d["Year"] == 1941: d["Country"] = "Germany"
    # And some entries sneak past the CountryIcon function.  This will get corrected manually as well.
    if d["Month"] == 2 and d["Day"] == "10" and d["Year"] == 1941: d["Country"] = "United Kingdom"
    # And some entries have the wrong icon for the description.
    if d["Month"] == 8 and d["Day"] == "14" and d["Year"] == 1942: d["Country"] = "Germany"
        
    # Using regex, we can get the description of the attack.  This may or may not be useful, but since we have
    # the description, why not use it?
    d["Details"]= CleanText(entry.getText())
    
    data.append(d)

# Remove the initial entry in data and add the column for success
del(data[0])

In [75]:
# Next, we'll add a column that indicates whether the attack was effective or not.
# THIS IS SUBJECTIVE BASED ON MY INTERPRETATION OF THE DESCRIPTION AND SUPPLEMENTAL READING!
# 1= effective, 0= not effective

# 1939
success= [1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1]
# 1940
success.extend([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1])
# 1941
success.extend([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1])
# 1942
success.extend([1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0])
# 1943
success.extend([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])
# 1944
success.extend([0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
# 1945
success.extend([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [76]:
# Additionally, we'll need columns for the defending city and country, and also the lat/lon
location_data= pd.read_csv("WW2_Location_Data.csv")

In [77]:
# Convert data to a Pandas dataframe
df= pd.DataFrame(data)
df["Success"]= success
df["DefCity"]= location_data["City under attack"]
df["DefCountry"]= location_data["Country.1"]
df["Latitude"]= location_data["Latitude"]
df["Longitude"]= location_data["Longitude"]
df= df[["Success","Month","Day","Year","Country","DefCity","DefCountry","Latitude","Longitude","Details"]]

In [78]:
# I found an entry with an incorrect date.  5/15/1940 should be 5/15/1943.
df["Year"][16]= 1943

In [79]:
# Finally, convert the pandas dataframe to a .csv
df.to_csv("WW2_Bombing_Data.csv")